In [0]:
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, col, when
from pyspark.sql.functions import split, explode, count


#Chargement du fichier
spark
sc = spark.sparkContext
df= spark.read.json('s3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json')
display(df.select("data.*").orderBy(df["data.price"].cast("float").desc()).limit(20))

appid,categories,ccu,developer,discount,genre,header_image,initialprice,languages,name,negative,owners,platforms,positive,price,publisher,release_date,required_age,short_description,tags,type,website
1200520,"List(Multi-player, Single-player, Co-op, LAN Co-op)",0,Fury Games,0,Action,https://cdn.akamai.steamstatic.com/steam/apps/1200520/header.jpg?t=1637367369,99900,English,Ascent Free-Roaming VR Experience,0,"0 .. 20,000","List(false, false, true)",6,99900,Fury Games,2019/12/27,0,"A premium highly-immersive VR experience developed for haptic feedback via motion floors, body suits and climate control. Go it alone or play with up to four friends in LAN co-op to destroy the infestation at the FuriCorp Research facility.","List(null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 83, null, null, null, null, null, null, null, null, null, null, null, null, 8, null, null, 10, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 10, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 9, null, null, null, null, null, 7, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 10, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 5, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 6, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 16, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 10, null)",game,http://www.furygames.net
253670,List(Single-player),0,Aartform,0,Animation & Modeling,https://cdn.akamai.steamstatic.com/steam/apps/253670/header.jpg?t=1596786459,29990,English,Aartform Curvy 3D 3.0,13,"0 .. 20,000","List(false, false, true)",31,29990,Aartform,2013/11/12,0,"Curvy 3D 3.0 is an innovative sculpting program for Windows, ideal for first time 3D artists. Drawing is Fast: Curvy brings new speed to 3D modelling by letting you draw shapes straight into 3D. Sculpting is Easy: Shapes are easy to work with using Curvy's artist friendly tools.","List(null, null, null, null, null, null, null, null, null, null, null, null, null, n

Databricks visualization. Run in Databricks to view.

Quels sont les genres les plus représentés ?

Les genres les plus représentés sont: indie,action, casual,adventure...

In [0]:
# Sélectionner les données pertinentes à partir du DataFrame
data = df.select("data.*")

data_exploded = data.withColumn("genre", explode(split(col("genre"), ", ")))
# Regrouper les données par genre et compter le nombre d'occurrences de chaque genre
genre_counts = data_exploded.groupBy("genre").count().orderBy(F.desc("count"))

# Afficher les genres les plus représentés
display(genre_counts)

Existe-t-il des genres qui ont un meilleur ratio d'avis positifs/négatifs ?

Les genres avec un meilleur ratio d'avis positifs: Photo Editing, Animation & Modeling, Design & Illustration.

Les genres avec un meilleur ratio d'avis négatifs: Nudity, Accounting, Movie.

In [0]:
from pyspark.sql.functions import sum

data_exploded = data.withColumn("genre", explode(split(col("genre"), ", ")))

data_exploded = data_exploded.groupBy("genre").agg(sum("positive").alias("positive"),
                                                  sum("negative").alias("negative"))

#On doit faire le ratio des données de data positive et data négative pour avoir une moyenne
result = data_exploded.withColumn("ratio", when((col("positive") + col("negative")) == 0, 0.5)
                         .otherwise(col("positive") / (col("positive") + col("negative"))))
result = result.withColumn("total", col("positive") + col("negative"))

display(result.select("genre", "ratio", "positive", "negative", "total").orderBy(F.desc("ratio")))

Certains éditeurs ont-ils des genres favoris ?
Les éditeurs préfèrent les genres: Indie, casual, rpg.

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lower,trim

# Récupération le nombres de jeux qu'il y a par genre 
data_exploded = data.withColumn("genre", F.explode(F.split(F.col("genre"), ", ")))
game_per_publisher_genre = data_exploded.groupby(lower(trim(col("publisher"))).alias("publisher"), "genre").count()
#Récupère le nombre de jeux qu'il y a part publisher
all_game_per_publisher = data.groupBy(lower(trim(col("publisher"))).alias("publisher")).count()
#Rassemble les données que l'on a par genre et au total (en un seul dataframe)
game_per_publisher_genre = game_per_publisher_genre.join(all_game_per_publisher.select("publisher", col("count").alias("total_games")), on="publisher", how="inner")
#Calcul de total de ratio par count/ le total de jeux*100
game_per_publisher_genre = game_per_publisher_genre.withColumn("ratio", (col("count") / col("total_games")) * 100)

# Définir une fenêtre de partitionnement par éditeur et trier par ratio décroissant
#(Permet de sélectionner par publishier la ligne qui a le plus grand ratio)

windowSpec = Window.partitionBy("publisher").orderBy(col("ratio").desc())

# Ajouter une colonne de rang basée sur le tri du ratio décroissant
game_per_publisher_genre = game_per_publisher_genre.withColumn("rank", F.row_number().over(windowSpec))

# Filtrer les lignes avec un rang égal à 1 pour obtenir uniquement la ligne avec le ratio le plus élevé pour chaque éditeur
favorite_genre_per_publisher = game_per_publisher_genre.filter(col("rank") == 1)

# Afficher les résultats
display(favorite_genre_per_publisher.orderBy("publisher"))

publisher,genre,count,total_games,ratio,rank
,Indie,108,134,80.59701492537313,1
"""revday studio""",Indie,1,1,100.0,1
<eye>game,Simulation,1,1,100.0,1
'kill on sight' game development,Simulation,1,1,100.0,1
((no-end-parens studio,Casual,1,1,100.0,1
(none),Indie,1,2,50.0,1
(stcg) smoker the car game,Simulation,1,1,100.0,1
"++good games, gamechanger charity",Adventure,1,1,100.0,1
+7 software,Casual,1,1,100.0,1
"+mpact games, llc.",Adventure,1,1,100.0,1


Quels sont les genres les plus lucratifs ?
Les genres les plus lucratifs sont: Indie, Action, Adventure, Casual, Simulation.

In [0]:
data_exploded = data.withColumn("genre", explode(split(col("genre"), ", ")))
data_exploded = data_exploded.groupBy("genre").agg(sum("price").alias("total_price"))

display(data_exploded.orderBy("total_price", ascending=False))


genre,total_price
Indie,2.6063036E7
Action,1.8358769E7
Adventure,1.7158179E7
Casual,1.2383583E7
Simulation,9851654.0
Strategy,9157201.0
RPG,8621295.0
Early Access,5375795.0
Sports,2385529.0
Racing,1771645.0


Databricks visualization. Run in Databricks to view.